# Chad Rainfall Analysis 2011

This notebook will be analysing daily rainfall in Chad for 2011 using CHIRPS satellite data. 

Due to the fact that Chad is a semi-arid climate, we identify the thresholds as:

**Drought Periods**: Days where the 7 day rolling average rainfall falls below the 10th percentile threshold.  
**Extreme Rainfall Events**: Peaks of daily rainfall above the 90th percentile threshold.

The analysis is performed first on **N'Djamena** and then extended to the **entire country**.

Below is importing the libraries required to complete this data analysis as well as loading the CHIRPS daily global rainfall for 2011 and inspect its dimensions and variables.

In [19]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


ds = xr.open_dataset("C:\Projects\geodataprojects\data\chirps_global_2011.nc")
ds

<xarray.Dataset> Size: 841MB
Dimensions:    (latitude: 400, longitude: 1440, time: 365)
Coordinates:
  * latitude   (latitude) float32 2kB -49.88 -49.62 -49.38 ... 49.38 49.62 49.88
  * longitude  (longitude) float32 6kB -179.9 -179.6 -179.4 ... 179.6 179.9
  * time       (time) datetime64[ns] 3kB 2011-01-01 2011-01-02 ... 2011-12-31
Data variables:
    precip     (time, latitude, longitude) float32 841MB ...
Attributes: (12/15)
    Conventions:       CF-1.6
    title:             CHIRPS Version 2.0
    history:           created by Climate Hazards Group
    version:           Version 2.0
    date_created:      2015-10-07
    creator_name:      Pete Peterson
    ...                ...
    reference:         Funk, C.C., Peterson, P.J., Landsfeld, M.F., Pedreros,...
    comments:           time variable denotes the first day of the given day.
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/org/chg/products/CH...
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html
    faq:               http://chg-wiki.geog.ucsb.edu/wiki/CHIRPS_FAQ

## Subset Chad and N'Djamena

With the data of daily rainfall globally in 2011, I am going to subset **Chad** with the coordinates of 7°N to 24°N, 14°E to 24°E so that I can use the rain data for only the specific country and then select a coordinate point of 12.1°N, 15°E for **N'Djamena**. 

Due to the nature of the CHIRPS dataset, the selected values will have to be on the grid and so the coordinates have been adjusted accordingly, but still is within N'Djamena.

In [20]:
lat_min, lat_max = 7.0, 24.0
long_min, long_max = 14.0, 24.0

chad = ds.sel(
    latitude = slice(lat_min, lat_max),
    longitude = slice(long_min, long_max)
)

lat_point = chad.latitude.sel(latitude=12.125, method="nearest")
long_point = chad.longitude.sel(longitude=15.125, method="nearest")

ndjamena_rain = chad["precip"].sel(
    latitude = lat_point,
    longitude = long_point,
)

dataframe_ndjamena = ndjamena_rain.to_dataframe().reset_index()
total_days = len(dataframe_ndjamena)
dry_days = (dataframe_ndjamena["precip"] == 0).sum()
print(f"{dry_days}/{total_days} days have no rainfall ({dry_days/total_days*100:.1f}%)")

267/365 days have no rainfall (73.2%)


## Calculate 7-day rolling average and thresholds

A rolling average smooths daily variability to highlight sustained dry periods and the thresholds of **90%** and **10%** percentiles will allow for finding the potential risk zones.

In [21]:
dataframe_ndjamena["rainfall_7day_avg"] = dataframe_ndjamena["precip"].rolling(7, min_periods=1).mean()

drought_threshold = dataframe_ndjamena["rainfall_7day_avg"].quantile(0.10)
dataframe_ndjamena["is_drought"] = dataframe_ndjamena["rainfall_7day_avg"] < drought_threshold

extreme_threshold = dataframe_ndjamena["precip"].quantile(0.90)
dataframe_ndjamena["is_extreme"] = dataframe_ndjamena["precip"] > extreme_threshold